In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import random

# keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras import optimizers
from keras import models
from keras import layers
from keras.losses import SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator

# sklearn library
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report 

In [ ]:
! unzip "../input/dogs-vs-cats-redux-kernels-edition/train.zip" -d train
! unzip "../input/dogs-vs-cats-redux-kernels-edition/test.zip" -d test

In [ ]:
# labels
file = os.listdir("./train/train")
Labels = list(map(lambda x: x.split('.')[0], file))

# for filenames
f = Path("./train/train")
File_Path = list(f.glob(r"**/*.jpg"))

# dataframe
File_Path = pd.Series(File_Path).astype(str)
Labels = pd.Series(Labels)
df = pd.concat([File_Path,Labels],axis=1)
df.columns = ['filename', 'category']

In [ ]:
df.head()

In [ ]:
train_set, test_data = train_test_split(df, test_size= 1/3, random_state = 42)
train_data, val_data = train_test_split(train_set, test_size= 1/3, random_state = 42)


print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

In [ ]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

In [ ]:
img_size = (128, 128)

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [ ]:
train_gen = img_gen.flow_from_dataframe(
    train_data, 
    x_col='filename',
    y_col='category',
    target_size=img_size,
    class_mode='binary',
    batch_size=32,
    shuffle = False
)
validation_gen = img_gen.flow_from_dataframe(
    val_data, 
    x_col='filename',
    y_col='category',
    target_size=img_size,
    class_mode='binary',
    batch_size=32,
    shuffle = False
)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])


In [ ]:
model.summary()

In [ ]:
history = model.fit(train_gen,
                    validation_data = validation_gen,
                    validation_steps=50,
                    steps_per_epoch=100,
                    epochs = 50,
                    verbose=1)

In [ ]:
#accuracy
acc = history.history['acc']
val_acc = history.history['val_acc']

#loss
loss = history.history['loss'] 
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(10, 5))

# visualising Accuracy 
plt.subplot(2, 1, 1)

plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')

plt.ylabel('Accuracy') 
plt.title('Training and Validation Accuracy') 

